# Train example notebook

This notebook is used to implement the training of a neural network for classification of `Cloud`, `Edge`, `Good` images. <br> It is advisable to use this notebook to get practice and debug your code. To speed up the execution, once you are ready, you should move to a scripted version.

## 1. - Imports

Select `CUDA_VISIBLE_DEVICES` to the `Graphics Proceesing Unit (GPU)` index that you want to use to enable the use of GPU.

In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0" # GPU index

Enabling autoreload of different packages.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch 
import sys
sys.path.insert(1, os.path.join("..", "data"))
sys.path.insert(1, os.path.join("..", "utils"))
from data_utils import Dataset
from plot_utils import plot_image
from torch.utils.data import DataLoader

## 2. - Datasets

### 2.1 - Creating datasets

Now we read the images from the target directory `path_data`. Set `path_data` to the directory containing the `Cloud`, `Edge`, `Good` subfolders.  Moreover, it will automatically split the total dataset into the train, cross validation and test splits by using a pseudo-random splitting algorithm. You can reproduce the split by specifying the variable `seed`. **NB**:
- The train split contains 70% of the whole images.
- The valid splits contains 15% of the whole images.
- The test splits contains 15% of the whole images.
<br>**YOU MUST NOT CHANGE THE TEST SPLIT SIZE!!!**

In [4]:
# Path to the data folder (update the variable to your path).
path_data=os.path.join("..", "data")
# Seed value
seed=22

<img src="utilities/images/danger_icon.png" style="margin:auto"/>

**N.B** Make sure to have created a dataset split into the three directories `Cloud`, and `Good`, `Edge`. Otherwise, the next cell will **fail!** <br>


In [5]:
dataset=Dataset(path_data=path_data, seed=seed)
dataset.read_data()

Parsing class: Cloud: 141it [00:19,  7.29it/s]
Parsing class: Edge: 97it [00:10,  8.88it/s]
Parsing class: Good: 66it [00:08,  7.61it/s]


**Hint:** before proceeding, make sure that your `Edge`,`Cloud`, and `Good` samples are well enough among the `train`, `valid`,`test` splits. To print datasets statistics, run the next line.  Remember that the number of images in the different splits is distributed as described above. <br> If you are not happy with the data distribution, you can update the seed used and create a new dataset by rerunning the cell above. 

In [6]:
dataset.get_statistics()

,train,valid,test
cloud,99,23,19
edge,70,15,12
good,43,8,15


### 2.2. - Create data loaders.

The next lines will create a dataloader. A data loader is used to break the dataset into batches of a size `batch_size`. <br> This is useful to ensure that your dataset will fit into your memory and to create a "stochastic" implementation of gradient descent. <br> For more information, please, check: [data loader](https://www.educative.io/answers/what-is-pytorch-dataloader).<br>
Specify `batch_size` (**Hint**: use powers of 2. Typical values are between 8 and 64).

In [7]:
batch_size=32

In [8]:
# Train loader
train_loader = DataLoader(dataset.get_split("train"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Cross validation data loader
valid_loader = DataLoader(dataset.get_split("valid"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Test data loader
test_loader = DataLoader(dataset.get_split("test"), batch_size=batch_size, pin_memory=False, shuffle=True)

## 3 - Training

Now, it is your turn! Add your code below to load a Neural Network model, select optimizers, learning rate and perform training. <br>
Good luck!

In [43]:
classes = ('cloud', 'edge', 'good')

In [44]:
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid 
import torch.nn.functional as F 
import numpy as np
import matplotlib.pyplot as plt

def resize_tensor_images(images, size=(256, 256)):
    # Resize the batch of images
    return F.interpolate(images, size=size, mode='bilinear', align_corners=False)

def normalize_individual_image(image):
    # Calculate the mean and std for each channel of the image
    mean = image.mean(dim=[1, 2])
    std = image.std(dim=[1, 2])

    # Ensure std is not zero to avoid division by zero
    std = std.clamp(min=1e-9)

    # Normalize the image
    normalized_image = (image - mean[:, None, None]) / std[:, None, None]
    return normalized_image
    
def tensor_to_numpy(tensor):
    # Rescale the tensor to 0-1 range
    tensor = tensor - tensor.min()
    tensor = tensor / tensor.max()
    # Move the tensor to CPU if it's on GPU
    tensor = tensor.cpu()

    # Convert to numpy and transpose from CxHxW to HxWxC for visualization
    numpy_image = tensor.numpy()
    numpy_image = np.transpose(numpy_image, (1, 2, 0))

    return numpy_image
normalized_batches_TRL = []
normalized_batches_VAL = []
for batch in train_loader:
    images, labels = batch

    resized_images_TRL = resize_tensor_images(images)

    # Normalize each image in the batch
    normalized_images_TRL = torch.stack([normalize_individual_image(img) for img in resized_images_TRL])
    normalized_batches_TRL.append((normalized_images_TRL, labels))

for batch in valid_loader:
    images, labels = batch

    resized_images_VAL = resize_tensor_images(images)

    # Normalize each image in the batch
    normalized_images_VAL = torch.stack([normalize_individual_image(img) for img in resized_images_VAL])
    normalized_batches_VAL.append((normalized_images_VAL, labels))

# first_image_tensor = normalized_images[0]

# # Convert the tensor to a NumPy array
# first_image_numpy = tensor_to_numpy(first_image_tensor)

# Display the image
# plt.imshow(first_image_numpy)
# plt.axis('off')  # Remove axis markers
# plt.show()


In [45]:


import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(16 * 64 * 64, 32)  
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()



In [46]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [54]:
for epoch in range(40):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(normalized_batches_TRL, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        print(inputs)
        print(labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

tensor([[[[-0.5328, -0.5328, -0.5328,  ...,  0.1901,  0.1901,  0.2077],
          [-0.5152, -0.5152, -0.5152,  ...,  0.2430,  0.2253,  0.2253],
          [-0.5152, -0.5152, -0.5152,  ...,  0.3311,  0.3311,  0.3487],
          ...,
          [-1.3791, -1.3791, -1.3791,  ..., -1.3791, -1.3791, -1.3791],
          [-1.3791, -1.3791, -1.3791,  ..., -1.3791, -1.3791, -1.3791],
          [-1.3791, -1.3791, -1.3791,  ..., -1.3791, -1.3791, -1.3791]],

         [[-0.4544, -0.4544, -0.4442,  ...,  0.2916,  0.3018,  0.3120],
          [-0.4339, -0.4339, -0.4339,  ...,  0.3325,  0.3018,  0.3120],
          [-0.4339, -0.4237, -0.4237,  ...,  0.4347,  0.4244,  0.4347],
          ...,
          [-1.4558, -1.4558, -1.4558,  ..., -1.4558, -1.4558, -1.4558],
          [-1.4558, -1.4558, -1.4558,  ..., -1.4558, -1.4558, -1.4558],
          [-1.4558, -1.4558, -1.4558,  ..., -1.4558, -1.4558, -1.4558]],

         [[-0.3567, -0.3670, -0.3670,  ...,  0.2711,  0.2814,  0.2917],
          [-0.3670, -0.3567, -

In [55]:
PATH = './test1.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(normalized_batches_VAL)
images, labels = next(dataiter)

# print images
for i in range(len(images)):
    first_image_tensor = images[i]

    # Convert the tensor to a NumPy array
    first_image_numpy = tensor_to_numpy(first_image_tensor)

    # Display the image
    plt.imshow(first_image_numpy)
    plt.axis('off')  # Remove axis markers
    plt.show()


In [57]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [60]:
outputs = net(images)
print(outputs)

tensor([[-0.1045, -0.1567,  0.0722],
        [-0.1120, -0.1534,  0.0725],
        [-0.1183, -0.1455,  0.0695],
        [-0.1084, -0.1562,  0.0825],
        [-0.0943, -0.1729,  0.0798],
        [-0.0819, -0.1696,  0.0694],
        [-0.0804, -0.1818,  0.0775],
        [-0.0813, -0.1736,  0.0664],
        [-0.0609, -0.1839,  0.0729],
        [-0.1053, -0.1574,  0.0744],
        [-0.0564, -0.1858,  0.0600],
        [-0.1012, -0.1622,  0.0691],
        [-0.1035, -0.1594,  0.0760],
        [-0.0872, -0.1771,  0.0774],
        [-0.0505, -0.1777,  0.0850],
        [-0.0938, -0.1666,  0.0849],
        [-0.0799, -0.1831,  0.0720],
        [-0.0913, -0.1725,  0.0701],
        [-0.0691, -0.1842,  0.0833],
        [-0.0781, -0.1733,  0.0805],
        [-0.0853, -0.1642,  0.0718],
        [-0.0783, -0.1627,  0.0831],
        [-0.1236, -0.1484,  0.0648],
        [-0.0909, -0.1674,  0.0856],
        [-0.0976, -0.1610,  0.0711],
        [-0.0547, -0.1869,  0.0796],
        [-0.0964, -0.1738,  0.0819],
 

In [59]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(32)))

Predicted:  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good  good 
